### 随机生成字符串，长度可设置

In [8]:
import random
import string

def get_random_str(length=8):
    """
    生成一随机字符串,最长长度为62
    length 字符串长度
    """
    return ''.join([(string.ascii_letters+string.digits)[x] for x in random.sample(range(0,62), length)])


3md1fn


### 结巴分词样例

In [5]:
import jieba
seg_list = jieba.cut("优秀的人才有机会")
"/ ".join(seg_list)

'优秀/ 的/ 人才/ 有/ 机会'

### pymongo 操作

In [42]:
from pymongo import MongoClient
from bson.objectid import ObjectId

def to_objectid(id):
    """ 将str转换为OjectId """
    return ObjectId(id)

# 测试本地环境下mongodb
c = MongoClient()
print('连接状态：',c.test_database)
# Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

连接状态： Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_database')

数据库有： ['admin', 'hiso', 'local', 'ye_payment', 'ye_subscribe', 'ye_user']

现在的数据库名： hiso

该数据库下的集合： ['user', 'tt']

插入数据_id： 5937b3440d5b984f27e6921c


In [ ]:
#建立于MongoClient 的连接
client = MongoClient('localhost', 27017)

# 查看有多少数据库
dbs = client.database_names()
print('\n数据库有：', dbs)

# 连接到某个数据库
db = client.hiso # client['hiso']
print('\n现在的数据库名：', db.name)

col_names = db.collection_names()
print('\n该数据库下的集合：', col_names)

# 连接到某个集合,若库里没有会在插入数据时新建该collections
col = db.user # collections: user
rs = col.insert_one({'username': 'willschang'})
print('\n插入数据_id：', rs.inserted_id)


In [55]:
# find({query}, {fields}) fields里的0表示不返回字段信息 查询数据，数据以list形式转换输出
data = col.find({'username': 'willschang'}, {'_id':0, 'username': 1})

# 数据条数
counts = data.count()
print('\n返回数据量：', counts)
print('查询到结果：', list(data))

# find_one() 查询数据，数据以dict形式返回
print(col.find_one())


返回数据量： 2
查询到结果： [{'username': 'willschang'}, {'username': 'willschang'}]
{'_id': ObjectId('5937b2070d5b984f27e69219'), 'username': 'willschang'}


In [58]:
key = 'wier,wer,{"df":2}'
cmd, key, values = key.split(',', 2)
print(cmd, key, values)
# '1'*1000000

wier wer {"df":2}


In [7]:
import requests
import time

url = 'http://120.24.86.73/?A=WRITE,test1,%s'%('1'*1000)


In [1]:
from sklearn import preprocessing

enc = preprocessing.OneHotEncoder()

enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])

enc.transform([[0, 1, 3]]).toarray()

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

In [12]:
bank = '15060736692'
l = len(bank)
print(l)

t = bank[:l-8] + '****' + bank[l-4:]
print(bank)
print(t)

a = "\"test\""
print(a)
aa = a.strip('\"')
print(aa)

11
15060736692
150****6692
"test"
test


### 多线程

In [19]:
import concurrent
import random
def read(q):
        print('Get %s from queue.' % q)
        time.sleep(random.random())

def main():
    futures = set()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for q in (chr(ord('A')+i) for i in range(26)):
            future = executor.submit(read, q)
            futures.add(future)
    try:
        for future in concurrent.futures.as_completed(futures):
            err = future.exception()
            if err is not None:
                raise err
    except KeyboardInterrupt:
        print("stopped by hand")


main()

### 多进程

In [20]:
import concurrent
import random
def read(q):
        print('Get %s from queue.' % q)
        time.sleep(random.random())

def main():
    futures = set()
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for q in (chr(ord('A')+i) for i in range(26)):
            future = executor.submit(read, q)
            futures.add(future)
    try:
        for future in concurrent.futures.as_completed(futures):
            err = future.exception()
            if err is not None:
                raise err
    except KeyboardInterrupt:
        print("stopped by hand")


main()

### 拼音包 pypinyin

[pypinyin API](http://pypinyin.mozillazg.com/zh_CN/master/api.html)

In [6]:
from pypinyin import pinyin, lazy_pinyin
import pypinyin

pinyin('中心', style=pypinyin.FIRST_LETTER)
chi_spell = pypinyin.slug('中心', separator='', style=pypinyin.FIRST_LETTER)
print(chi_spell)
chi_spell_all = pypinyin.slug('中心', separator='')
print(chi_spell_all)

zx
zhongxin


### 

In [1]:
from collections import defaultdict

def find_anagram(input_data):
    """ find all the anagram from a dictionary
    """

    # initize a cotoutine
    def start(func):
        def _(*args, **kwargs):
            g = func(*args, **kwargs)
            g.send(None)
            return g
        return _

    @start
    def sign(target):
        while True:
            words = yield
            for w in words:
                target.send([''.join(sorted(w)), w])
            target.send(None)  # flag incicates that all data have been seen

    @start
    def sort(target):
        sign_words = []
        while True:
            word = yield
            if word:
                sign_words.append(word)
            else:  # all word have sort
                target.send(sorted(sign_words))

    @start
    def squash():
        nonlocal dictionary  # python3 only: use the variable define in outside
        while True:
            word_list = yield
            for x in word_list:
                dictionary[x[0]].add(x[1])

    dictionary = defaultdict(set)

    sign(sort(squash())).send(input_data)

    # filter the word has no anagram
    return filter(lambda x: len(x[1]) > 1, dictionary.items())


if __name__ == "__main__":

    test_data = ['abc', 'acb', 'bca', 'iterl', 'liter', 'hello',
                 'subessential', 'suitableness', 'hello']
    result = find_anagram(test_data)
    for each in result:
        print(each[0], ':', each[1])

eilrt : {'iterl', 'liter'}
abeeilnssstu : {'subessential', 'suitableness'}
abc : {'bca', 'acb', 'abc'}


## 两个dict合并

In [3]:
import itertools

#for c in itertools.chain('ABC', 'XYZ'):
    #print c
    
a = {'a':1, 'b':3}
b = {'c':'test', 'd': 'hi'}
c = dict(itertools.chain(a.items(), b.items()))
print c

cc = dict(a, **b)
print cc

{'a': 1, 'c': 'test', 'b': 3, 'd': 'hi'}
{'a': 1, 'c': 'test', 'b': 3, 'd': 'hi'}


### 时间 处理

In [4]:
from datetime import datetime, date
import time

n = datetime.now().date()
print n
nn = date(2012,11,19)
print nn

print date.today().strftime('%Y%m%d')

tr = datetime.now().replace(hour=6, minute=30, second=0, microsecond=0)

print tr
trt = tr.timetuple()
print trt

tt = time.mktime(trt)
print tt 
print tt + 28800
print time.time()

d = datetime.fromtimestamp(time.time())  
str1 = d.strftime("%Y-%m-%d %H:%M:%S")
print type(str1)

2017-09-19
2012-11-19
20170919
2017-09-19 06:30:00
time.struct_time(tm_year=2017, tm_mon=9, tm_mday=19, tm_hour=6, tm_min=30, tm_sec=0, tm_wday=1, tm_yday=262, tm_isdst=-1)
1505773800.0
1505802600.0
1505813742.9
<type 'str'>
